# LR Classification

Classification, a task popular in machine learning, determines whether and how a model can distinguish between sets of text.

It works like this. Everyone with email relies on classification to separate spam from legitimate emails. Email providers train classification models to recognize the difference by giving them emails they have labeled “spam” and “not spam.” They then ask the model to learn the features that most reliably distinguish the two types, which could include a preponderance of all caps or phrases like “free money” or “get paid.” They test the model by giving it unlabeled emails and asking it to classify them. If the model can do it accurately a high percentage of the time, that’s a good spam filter.

We can take the underlying idea and apply it to many experiments.

## Today's experiment

We are going to use a corpus of obituaries from _The New York Times_ in order to test whether our model can learn to distinguish between obituaries about men and women.

## Imports

As always, we begin with some imports.

In [ ]:
import pandas as pd
import glob
from pathlib import Path
from pandas import DataFrame
from pandas import Series, DataFrame
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import pearsonr, norm

## Dataset

For this notebook, we'll be using a dataset of _New York Times_ obituaries. Let's import the gdown library so that we can load in the dataset and unzip it:

In [ ]:
# For downloading large files from Google Drive
import gdown

# Download the zip file
gdown.download('https://drive.google.com/uc?export=download&id=1G0Aeg8dzZGPOCNFZ77U-s9CfEnR8efbB', quiet=False)

In [ ]:
# unzip it
!unzip NYT-Obituaries.zip

In [ ]:
# collect filepaths as files
directory = "./NYT-Obituaries/"
files = glob.glob(f"{directory}/*.txt")

len(files)

In [ ]:
# and collect obit titles, which are also the final section of the filepaths
obit_titles = [Path(file).stem for file in files]
obit_titles

## Create document-term matrix

### Initiate CountVectorizer as vectorizer

Remember document-term matrices, aka doc-term matrices, aka dtms? We learned about them in our tf-idf notebooks. Our classifier uses a dtm as its input. We build it with scikit-learn's CountVectorizer, which we already imported up above.

When we load our vectorizer, we include an argument to encode as utf-8 and we load our stopwords. In this case, we're using a custom stopwords list rather than the default sklearn one. You may end up using a custom stopwords list in your final projects.

In addition, we can set the minimum number of times a word must appear in the corpus for it to be included in the dtm. In this case, I've set it at 20.

In [ ]:
# another sklearn library to help load stopwords
from sklearn.feature_extraction import text

# Download the stopwords file
gdown.download('https://drive.google.com/uc?export=download&id=1BQ8zVSiG_WKpXNB81Y1P9yyi2L43UeJD', quiet=False)

# open it
text_file = open('./jockers_stopwords.txt')
jockers_words = text_file.read().split()
new_stopwords = list(text.ENGLISH_STOP_WORDS.union(jockers_words))

# create dtm
corpus_path = './NYT-Obituaries/'
vectorizer = CountVectorizer(input='filename',
                             encoding='utf8',
                             stop_words = new_stopwords,
                             min_df=20, dtype='float64')

### Make list of filepaths

If you recall, CountVectorizer builds a dtm from a list of filepaths. So we will provide that:

In [ ]:
corpus = []
for title in obit_titles:
    filename = title + ".txt"
    corpus.append(corpus_path + filename)
dtm = vectorizer.fit_transform(corpus)

### Get feature names and set as column titles

The columns store word counts. We want to name the columns with the words stored in each, and to transform the dtm into a pandas dataframe, as follows:

In [ ]:
features = vectorizer.get_feature_names_out()
dtm_array = dtm.toarray()
dtm_df = DataFrame(dtm_array, columns=features)
print('df shape is: ' + str(df.shape))

dtm_df

Our dataframee has 378 rows, one for each document, or obituary, and 2985 columns, one for each word that's not in stopwords and appears at least 20 times in the corpus.

### Pandas interlude ###

I re-found [this](https://twitter.com/mmitchell_ai/status/1454931443386228751) on Twitter the other night, posted by Dr. Margaret Mitchell, the former co-lead of Google's EthicalAI group and now Big Deal at HuggingFace (the folx behind the transformer libraries we'll be using next week):

<img src="http://lklein.com/wp-content/uploads/2021/11/Screen-Shot-2021-11-01-at-10.14.24-AM.png" width=500px>

In any case, now it's time to import our metadata:

## Import metadata

In [ ]:
gdown.download('https://drive.google.com/uc?export=download&id=1Pca0n-vWTy_FcF0oKWt9iHwxuBrDnxfd', quiet=False)

metadata = pd.read_csv("./NYT-Obituaries.csv", encoding = 'utf-8')
metadata = metadata.rename(columns={'title': 'obit_title'})
metadata = metadata[["obit_title", "gender", "date"]]
metadata

Our metadata is stored as a pandas dataframe with a row for each obituary and three columns: title, gender, and year.

Let's now concatinate the dtm to it so that everything is in one place.

## Concatenate metadata and doc-term dataframe

We'll use the pandas `concat` methods, specifying that the data should be concatinated as additional columns (that's the `axis = 1` parameter. (The default would be `0` to concatinate as additional rows.)

In [ ]:
df_concat = pd.concat([metadata, dtm_df], axis = 1)

In [ ]:
df_concat.head()

## Equalize numbers of men and women

We want our dataframe to have equal numbers of men and women. How many women are there? Women are counted as 1 and men as 0, so if we sum the gender column, we'll have the number of women:

In [ ]:
metadata['gender'].sum()

Then we separate men and women into two dataframes and take a random sample of 93 obituaries about men. Then we concatenate the sampled men dataframe with the original women dataframe and reset the index.

In [ ]:
# separate by gender
df_men = df_concat[df_concat['gender'] == 0]
df_women = df_concat[df_concat['gender'] == 1]

# sample 93 of the men
df_93_men = df_men.sample(n=93)

# concatenate the 93 men and 93 women
df_final = pd.concat([df_93_men, df_women])

# reset the index
df_final = df_final.reset_index()
df_final = df_final.drop(columns="index")

df_final



### Match metadata and data dataframes with subset of df_final

We'll continue to use the isolated metadata associated with our subsampled dataset, as well as the corresponding dtm, so we need to ensure they match our subsetted df_final dataframe

In [ ]:
metadata = df_final[["obit_title", "gender", "date"]]
metadata

In [ ]:
dtm_df = df_final.loc[:,'000':]

dtm_df

## Let's run our classifier!

Once we have a dataframe with metadata and feature counts we're ready to run our classifier!

### We add columns to hold the probabilities and predicted class to our metadata

As we run the model, we are going to store its output with our metadata. This will allow us to easily examine the model's output.

In [ ]:
metadata.loc[:, 'PROBS'] = np.nan
metadata.loc[:, 'PREDICTED'] = np.nan

metadata

### Load model

We will use scikit-learn's `LogisticRegression` model. There are many other options for classifier models. Some are better for some tasks, other for others. LogisticRegression is standard for classifying literature. We set the penalty as L1 (Lasso / least absolute deviations) and the 'C' value as 1.0.

If you decide to specialize in classification, you can explore further the implications of these regularization parameters. If you're curious now, [here is a nice intuitive primer](https://medium.com/analytics-vidhya/l1-vs-l2-regularization-which-is-better-d01068e6658c) on L1 vs the other main option, L2 (Ridge Regression). (In sk-learn, there is also an option in the middle, the Elastic-Net, which leads to an amount of sparsity between that of L1 and L2.)

As far as C, large values of C (like ours) give more freedom to the model. Conversely, smaller values of C (e.g. .1 or .01) constrain the model more.

`class_weight` adjusts the weights of classes inversely proportional to their frequencies in the data. This is particularly useful for imbalanced datasets, where one class may dominate. The model will give more weight to the minority class to handle imbalances.

The `solver` is used to optimize the model. 'liblinear' is a good choice when using L1 regularization, and it's suitable for smaller datasets as it uses coordinate descent.

In [ ]:
model = LogisticRegression(penalty = 'l1',
                           C = 1.0,
                           class_weight='balanced',
                           solver='liblinear')

## Run the model!

We run the model in the following for-loop.

Classification models need classes: they need the texts grouped into different sets. Our metadata has built-in classes: gender. Men are stored as 0; women as 1. We could, if we wanted, create a new 0/1 class based on something else.

Each iteration trains on all the obits except one, then predicts which class the excluded obit belongs to. This is called leave-one-out classification. It's helpful when you're working with a small dataset. There are other ways of dividing training and testing sets, including the 80/20 split we used lass class with our book reviews genres classifier.

In [ ]:
weights = None  # initialize weights variable for future use

for index, row in dtm_df.iterrows():
  print("Obit #" + str(index)) # keep track of where we are in the corpus

  X_train = dtm_df.drop(index)    # create training set X-train that includes all but current row (index)
  y_train = metadata['gender'].drop(index)  # create labels for training set (y_train) that includes all labels but current row (index)
  X_test = pd.DataFrame([row])    # create test set of just the current row
  y_test = metadata['gender'][index] # extracts actual gender label of current row for future comparison

  model.fit(X_train, y_train) # fit the model using the training data (X_train) and associated labels (y_train)
  y_pred = model.predict(X_test)[0] # create predictions on the test set, which is the current row (X_test)

  # print out some status messages
  print(f"{metadata['obit_title'][index]}, actual gender: {y_test}, predicted gender: {y_pred}")

  metadata.loc[index, 'PREDICTED'] = y_pred # store predicted gender
  metadata.loc[index, 'PROBS'] = model.predict_proba(X_test)[0][1] # caculate probability of predicted class (gender)

  # keep track of accumulted feature weights over time
  # later, we will average these to get the average feature weights
  if weights is None:
    weights = model.coef_[0]
  else:
    weights += model.coef_[0]



Let's take a look at our results as stored in the `metadata` dataframe.

In [ ]:
metadata

## Check accuracy

It's still a little hard to get an overall sense of things, so let's calculate overall accuracy.

In [ ]:
# clean up brackets
metadata = metadata.replace([0], 0)
metadata = metadata.replace([1], 1)

# add result column
sum_column = metadata['gender'] - metadata['PREDICTED']
metadata['RESULT'] = sum_column

# check accuracy
metadata_correct = metadata[metadata['RESULT'] == 0]
accuracy = len(metadata_correct) / len(metadata)
print("Model accuracy: " + str(accuracy))

Hm... not a very good classifier! At random, the model should guess correctly 50% of the times.

## Plot a confusion matrix

Let's just take a look at a confusion matrix to see if anything interesting is going on before we move on. Here's some code to do that.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

actual = metadata['gender'].array
predicted = metadata['PREDICTED'].array

cm = confusion_matrix(actual, predicted)

print(cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['man','woman'])

disp.plot()
plt.show()

## Calculate precision and recall

We're going to move on soon, but so you have it, here's how we calculate precision and recall in this context:

In [ ]:
from sklearn.metrics import precision_score, recall_score

y_true = metadata['gender']
y_pred = metadata['PREDICTED']

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

print("Precision:", precision)
print("Recall:", recall)


## Examining features

Sometimes it's not as much the overall accuracy of the classifier that's interesting as much as the features themselves. Let's explore which ones most help the model make its classification decision.

In this case, the features tell us which words are most likely to tip off the model that a given obituary is about a man or that another is about a woman.

### Calculating p-values

This part should be familiar to many of you. We'll define a Z-test function and use it to calculate p-values. We'll use these to determine the statistical significance of individual features.

(Our null hypothesis here, as in all z-tests, is a normal distribution.)

Note: As many of you might know, p-values have come under scrutiny in recent years as a measure of significance. The standard threshold for signficance (0.05) is arbitrary. Its meaning is debated. But its authority for a long time incentivized what came to be called p-hacking: the practice of manipulating ones work so it would pass the 0.05 threshold and count as significant. Very recently statisticians and other scholars have argued for abandoning the term "statistical significance," arguing that it reduces the complexity of determining whether a given result is meaningful in context. tl;dr: don't put too much stock in significance; consider the holistic context of results

In [ ]:
def Ztest(vec1, vec2):
    # calculate means, standard deviations, and lengths of the two data vectors
    X1, X2 = np.mean(vec1), np.mean(vec2)
    sd1, sd2 = np.std(vec1, ddof=1), np.std(vec2, ddof=1)
    n1, n2 = len(vec1), len(vec2)

    # calculate pooled standard error
    pooledSE = np.sqrt(sd1**2/n1 + sd2**2/n2)

    # compute Z statistic
    z = (X1 - X2) / pooledSE

    # compute p-value
    pval = 2 * norm.sf(abs(z))

    return z, pval

Now let's use it to caculate p-values for each of the features using our `metadata` and `dtm_df` dataframes.

In [ ]:
feature_data = []  # create a list to store feature information

# calculate average feature weights
avg_weights = weights / len(dtm_df)

for feature in dtm_df.columns:
  # separate data for men and women based on the current feature.
  men_feature_data = dtm_df.loc[metadata['gender'] == 0, feature]
  women_feature_data = dtm_df.loc[metadata['gender'] == 1, feature]

  # calculate z-test and p-value using the Ztest function.
  z_score, p_value = Ztest(men_feature_data, women_feature_data)

  # append feature data to the list
  feature_data.append({
    'feature': feature,
    'p_value': p_value,
    'avg_weight': avg_weights[dtm_df.columns.get_loc(feature)]
  })

# convert the list of dictionaries into a DataFrame
feature_df = pd.DataFrame(feature_data)

# sort by average weight then print
feature_df = feature_df.sort_values('avg_weight', ascending = True)

feature_df


This is more interesting! The negative LR weights tell the model that the obituary is likely about a woman. The positive LR weights tell the model the obituary is likely about a man. Let's take a slightly closer look.

In [ ]:
print("Words associated with women:")
feature_df.sort_values('avg_weight', ascending = True).head(20)


What can you infer from these words about the coverage of famous women in their obituaries in the NYT?

In [ ]:
print("Words associated with men:")
feature_df.sort_values('avg_weight', ascending = False).head(20)

What can you infer from these words about the coverage of famous men in their obituaries in the NYT?

## Filtering by p-value

Finally, we can filter further by setting a p-value threshold. For our purposes, we can set a high threshold, which to normalize we need to divide by the number of features

Remember significance is a contested concept. What most matters is understanding the meaning of numbers in context. For us, any feature with a logistic regression weight gives us useful information, and p-values help us understand just how robust that feature is.

In [ ]:
# to account for the many features / mulitple hypothesis tests
# sig_thresh = 50.00 / len(feature_df)

sig_thresh = .05
print("Significance threshold: " + str(sig_thresh))

# filter by p-values that pass that threshold
out = feature_df[(feature_df['p_value'] <= sig_thresh)].sort_values('avg_weight', ascending = True)
out = out[out['avg_weight'] != 0]
outM = out[out['avg_weight'] >= 0]
outW = out[out['avg_weight'] <= 0]

# pass remaining features to list and print them out
outM = outM['feature'].tolist()
print("Here are significant words that distinguish men: " + str(outM))

outW = outW['feature'].tolist()
print("Here are significant words that distinguish women: " + str(outW))

## And that's it!